In [38]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression as LR
from matplotlib import pyplot as plt
from tools import *
from sklearn.decomposition import PCA
import plotly.express as px
from sklearn.linear_model import LinearRegression, Ridge, Lasso
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error as MSE
import torch
from torch.utils import data
from torch import nn
from sklearn.preprocessing import PolynomialFeatures
from scipy.optimize import LinearConstraint, NonlinearConstraint, Bounds, minimize

# Load Data

In [13]:
def getObs(n):
    obs = {}
    for i in range(n):
        try:
            df = pd.read_csv(f'rocket-results/{i}.csv')
        except:
            print('Missing', i)
        obs[i] = df
    return obs

In [14]:
def initializeConfig():
    configs = {}
    with open('sample_list.pkl', 'rb') as f:
        d = pickle.load(f)
    configs.update(d)
    with open('sample_list_100.pkl', 'rb') as f:
        d = pickle.load(f)
    configs.update(d)
    with open('sample_list_200.pkl', 'rb') as f:
        d = pickle.load(f)
    configs.update(d)
    with open('sample_list_300.pkl', 'rb') as f:
        d = pickle.load(f)
    configs.update(d)
    return configs

In [15]:
config = initializeConfig()
obs = getObs(400)

In [16]:
data = {}
for i in range(len(config)):
    data[i] = list(config[i]['S'].values()) + list(config[i]['B'].values())
    data[i].append(obs[i].max()['Altitude (ft)'])
    data[i].append(obs[i].mean()['Stability Margin (cal)'])
    data[i].append(obs[i].max()['Time (sec)'])
df = pd.DataFrame.from_dict(data, orient='index')
df.iloc[284]

/var/folders/ff/y3gj2gld0zqfjtq_lhxz03t00000gn/T/ipykernel_30584/2135078581.py:5: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  data[i].append(obs[i].mean()['Stability Margin (cal)'])


0         3.734223
1         7.875106
2         2.509849
3         7.372139
4         8.753891
5         7.977354
6         7.539283
7         8.558976
8     32187.640000
9         5.071216
10       99.001530
Name: 284, dtype: float64

In [17]:
df = df.rename(columns={0: "Schord", 1: "Sspan", 2: "Ssweep", 3: "Stip", 4: "Bchord", 
                        5: "Bspan", 6: "Bsweep", 7: "Btip", 8: "Altitude", 9: "Stability", 10: "Time"})
df['Schordspan'] = df['Schord'] * df['Sspan']
df['Schordsweep'] = df['Schord'] * df['Ssweep']
df['Schordtip'] = df['Schord'] * df['Stip']
df['Sspansweep'] = df['Sspan'] * df['Ssweep']
df['SSpantip'] = df['Sspan'] * df['Stip']
df['SSweeptip'] = df['Ssweep'] * df['Stip']

df['Bchordspan'] = df['Bchord'] * df['Bspan']
df['Bchordsweep'] = df['Bchord'] * df['Bsweep']
df['Bchordtip'] = df['Bchord'] * df['Btip']
df['Bspansweep'] = df['Bspan'] * df['Bsweep']
df['BSpantip'] = df['Bspan'] * df['Btip']
df['BSweeptip'] = df['Bsweep'] * df['Btip']
alt = df.pop("Altitude")
stab = df.pop("Stability")
time = df.pop("Time")

df.insert(len(df.columns), "Altitude", alt)
df.insert(len(df.columns), "Stability", stab)
df.insert(len(df.columns), "Time", time)
df

,Schord,Sspan,Ssweep,Stip,Bchord,Bspan,Bsweep,Btip,Schordspan,Schordsweep,...,SSweeptip,Bchordspan,Bchordsweep,Bchordtip,Bspansweep,BSpantip,BSweeptip,Altitude,Stability,Time
0,9.465233,9.191411,6.313146,6.181568,4.253219,3.362766,2.004250,3.767645,86.998844,59.755399,...,39.025144,14.302581,8.524515,16.024619,6.739826,12.669711,7.551304,0.004903,-5.121752,0.0100
1,5.050903,2.807665,7.250966,4.202998,7.871765,6.329711,3.563606,6.148326,14.181241,36.623926,...,30.475799,49.826002,28.051867,48.398181,22.556595,38.917130,21.910210,77117.160000,1.278655,154.9973
2,8.796076,5.131824,8.469040,8.168441,6.226139,7.792487,6.816335,8.795342,45.139920,74.494323,...,69.178850,48.517110,42.439452,54.761021,53.116208,68.537590,59.951999,51216.000000,5.212006,125.0071
3,5.590219,9.233072,6.352605,8.429602,5.672564,5.705322,5.717624,3.420081,51.614896,35.512459,...,53.549937,32.363803,32.433584,19.400628,32.620884,19.512664,19.554736,0.004691,-2.489141,0.0100
4,2.916229,4.216994,4.711545,6.462613,4.887217,2.638681,9.605809,5.234709,12.297724,13.739946,...,30.448892,12.895807,46.945675,25.583158,25.346667,13.812727,50.283615,0.004818,-4.231226,0.0100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,2.608430,3.346465,5.201403,9.166944,6.192108,3.850187,7.960717,8.586406,8.729021,13.567499,...,47.680974,23.840776,49.293623,53.167952,30.650253,33.059270,68.353950,70830.750000,1.673402,149.0006
396,7.801407,2.628427,5.543288,9.133362,7.014197,5.730589,4.246228,6.930445,20.505433,43.245446,...,50.628851,40.195478,29.783881,48.611503,24.333390,39.715531,29.428252,72485.310000,1.614970,150.9995
397,7.364198,8.455093,4.617628,3.805883,4.280772,6.398586,8.552977,8.138172,62.264979,34.005123,...,17.574150,27.390888,36.613344,34.837657,54.726959,52.072793,69.605594,0.000000,-1.734951,0.0000
398,7.094305,2.423077,2.934079,6.637124,4.526698,3.664447,4.901074,5.327160,17.190049,20.815255,...,19.473850,16.587847,22.185683,24.114444,17.959727,19.521095,26.108803,73446.170000,0.853824,150.9995


In [21]:
filtered_df = df[df['Time'] > 90]
filtered_df

,Schord,Sspan,Ssweep,Stip,Bchord,Bspan,Bsweep,Btip,Schordspan,Schordsweep,...,SSweeptip,Bchordspan,Bchordsweep,Bchordtip,Bspansweep,BSpantip,BSweeptip,Altitude,Stability,Time
1,5.050903,2.807665,7.250966,4.202998,7.871765,6.329711,3.563606,6.148326,14.181241,36.623926,...,30.475799,49.826002,28.051867,48.398181,22.556595,38.917130,21.910210,77117.16,1.278655,154.9973
2,8.796076,5.131824,8.469040,8.168441,6.226139,7.792487,6.816335,8.795342,45.139920,74.494323,...,69.178850,48.517110,42.439452,54.761021,53.116208,68.537590,59.951999,51216.00,5.212006,125.0071
5,8.562160,4.337768,7.930397,4.372560,9.417980,8.246008,2.728282,6.326085,37.140668,67.901332,...,34.676133,77.660732,25.694900,59.578937,22.497430,52.164942,17.259340,54082.50,4.485508,129.0079
6,8.703811,2.188438,9.143090,2.635864,9.087634,8.094306,6.826608,3.754130,19.047753,79.579730,...,24.099946,73.558093,62.037714,34.116158,55.256655,30.387077,25.627973,78784.51,0.613785,156.9962
8,8.074672,3.479253,5.061755,2.201202,8.430403,6.003352,2.080965,3.394916,28.093824,40.872014,...,11.141944,50.610675,17.543371,28.620512,12.492762,20.380874,7.064700,59576.45,2.920013,136.0078
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
393,5.247843,7.799644,8.441005,9.658211,6.937770,7.882306,6.353957,9.196173,40.931306,44.297067,...,81.524999,54.685626,44.082290,63.800932,50.083829,72.487046,58.432082,39893.78,5.851534,111.0041
395,2.608430,3.346465,5.201403,9.166944,6.192108,3.850187,7.960717,8.586406,8.729021,13.567499,...,47.680974,23.840776,49.293623,53.167952,30.650253,33.059270,68.353950,70830.75,1.673402,149.0006
396,7.801407,2.628427,5.543288,9.133362,7.014197,5.730589,4.246228,6.930445,20.505433,43.245446,...,50.628851,40.195478,29.783881,48.611503,24.333390,39.715531,29.428252,72485.31,1.614970,150.9995
398,7.094305,2.423077,2.934079,6.637124,4.526698,3.664447,4.901074,5.327160,17.190049,20.815255,...,19.473850,16.587847,22.185683,24.114444,17.959727,19.521095,26.108803,73446.17,0.853824,150.9995


# NN Stability Classifier

In [18]:
class Classifier(torch.nn.Module):

    def __init__(self):
        super(Classifier, self).__init__()

        self.linear1 = torch.nn.Linear(45, 90)
        self.activation1 = torch.nn.ReLU()
        self.linear2 = torch.nn.Linear(90, 60)
        self.activation2 = torch.nn.ReLU()
        self.linear3 = torch.nn.Linear(60, 30)
        self.activation3 = torch.nn.ReLU()
        self.linear4 = torch.nn.Linear(30, 10)
        self.activation4 = torch.nn.ReLU()
        self.linear5 = torch.nn.Linear(10, 1)

    def forward(self, x):
        
        x = self.linear1(x)
        x = self.activation1(x)
        x = self.linear2(x)
        x = self.activation2(x)
        x = self.linear3(x)
        x = self.activation3(x)
        x = self.linear4(x)
        x = self.activation4(x)
        x = self.linear5(x)
        
        return torch.sigmoid(x)

In [22]:
def load_arrays(data_arrays, batch_size, train = True):
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle = train)

In [181]:
classif = Classifier()
classif.double()

poly = PolynomialFeatures(2)

raw_X = df.iloc[:,:8]
raw_Y = df.iloc[:,-1] > 90
batch_size = 10
idxs = np.array(list(range(0, 300)))# + list(range(0))) # FOR CV

train_X = torch.from_numpy(poly.fit_transform(raw_X.iloc[idxs])).double()
train_Y = torch.from_numpy(np.array([[x] for x in raw_Y.iloc[idxs]])).double()

test_X = torch.from_numpy(poly.fit_transform(raw_X[300:400])).double()
test_Y = torch.from_numpy(np.array([[x] for x in raw_Y[300:400]])).double()
data_iter = load_arrays((train_X, train_Y), 10)

In [182]:
loss = nn.BCELoss()
trainer = torch.optim.SGD(classif.parameters(), lr=0.01)
num_epochs = 250
def train(num_epochs, trainer, loss):
    for epoch in range(num_epochs):

        for X, y in data_iter:
            X.requires_grad = True
            l = loss(classif(X) ,y)

            trainer.zero_grad() #sets gradients to zero

            l.backward() # back propagation

            trainer.step() # parameter update

        l = loss(classif(train_X), train_Y)
        if epoch % 50 == 0:
            print(f'epoch {epoch + 1}, loss {l:f}')
            #print(X.grad)

In [183]:
train(num_epochs, trainer, loss), sum((classif(test_X) >= 0.5) == test_Y)/len(test_Y)

epoch 1, loss 0.527873
epoch 51, loss 0.034600
epoch 101, loss 0.005178
epoch 151, loss 0.001587
epoch 201, loss 0.000876


(None, tensor([0.8800]))

# Altitude Regression

In [195]:
poly = PolynomialFeatures(2)
raw = filtered_df.iloc[:,:8]
qr = LinearRegression()
idxs = np.array(list(range(150)))# + list(range(50)))

ridge = Ridge(alpha=0.6)
ridge.fit(poly.fit_transform(raw.iloc[idxs]), filtered_df.iloc[idxs, -3])
lasso = Lasso(alpha=.2)
lasso.fit(poly.fit_transform(raw.iloc[idxs]), filtered_df.iloc[idxs, -3])

qr.fit(poly.fit_transform(raw.iloc[idxs]), filtered_df.iloc[idxs, -3])
print('R2', ridge.score(poly.fit_transform(raw.iloc[idxs]), filtered_df.iloc[idxs, -3]))
print('Test RMSE Quad', np.sqrt(MSE(filtered_df.iloc[150:, -3], qr.predict(poly.fit_transform(raw[150:])))))
print('Test RMSE ridge', np.sqrt(MSE(filtered_df.iloc[150:, -3], ridge.predict(poly.fit_transform(raw[150:])))))
print('Test RMSE lasso', np.sqrt(MSE(filtered_df.iloc[150:, -3], lasso.predict(poly.fit_transform(raw[150:])))))

print('Train RMSE', np.sqrt(MSE(filtered_df.iloc[idxs, -3], ridge.predict(poly.fit_transform(raw.iloc[idxs])))))

R2 0.9929351496740758
Test RMSE Quad 1815.417729777189
Test RMSE ridge 2056.5256021326427
Test RMSE lasso 1815.019990798251
Train RMSE 1186.314188487517


# Solving COP

In [86]:
def stab_constraint(X):
    with torch.no_grad():
        return classif(torch.from_numpy(poly.fit_transform([X])))[0] - 0.6
def l0(X):
    return X[0]-2
def l1(X):
    return X[1]-2
def l2(X):
    return X[2]-2
def l3(X):
    return X[3]-2
def l4(X):
    return X[4]-2
def l5(X):
    return X[5]-2
def l6(X):
    return X[6]-2
def l7(X):
    return X[7]-2

def u0(X):
    return 10-X[0]
def u1(X):
    return 10-X[1]
def u2(X):
    return 10-X[2]
def u3(X):
    return 10-X[3]
def u4(X):
    return 10-X[4]
def u5(X):
    return 10-X[5]
def u6(X):
    return 10-X[6]
def u7(X):
    return 10-X[7]
def objective(X):
    return -ridge.predict(poly.fit_transform([X]))[0]
classif.eval()

Classifier(
  (linear1): Linear(in_features=45, out_features=90, bias=True)
  (activation1): ReLU()
  (linear2): Linear(in_features=90, out_features=60, bias=True)
  (activation2): ReLU()
  (linear3): Linear(in_features=60, out_features=30, bias=True)
  (activation3): ReLU()
  (linear4): Linear(in_features=30, out_features=10, bias=True)
  (activation4): ReLU()
  (linear5): Linear(in_features=10, out_features=1, bias=True)
)

In [111]:
ineq_cons = {'type': 'ineq',
             'fun' : stab_constraint}
L0 = {'type': 'ineq',
             'fun' : l0}
L1 = {'type': 'ineq',
             'fun' : l1}
L2 = {'type': 'ineq',
             'fun' : l2}
L3 = {'type': 'ineq',
             'fun' : l3}
L4 = {'type': 'ineq',
             'fun' : l4}
L5 = {'type': 'ineq',
             'fun' : l5}
L6 = {'type': 'ineq',
             'fun' : l6}
L7 = {'type': 'ineq',
             'fun' : l7}
U0 = {'type': 'ineq',
             'fun' : u0}
U1 = {'type': 'ineq',
             'fun' : u1}
U2 = {'type': 'ineq',
             'fun' : u2}
U3 = {'type': 'ineq',
             'fun' : u3}
U4 = {'type': 'ineq',
             'fun' : u4}
U5 = {'type': 'ineq',
             'fun' : u5}
U6 = {'type': 'ineq',
             'fun' : u6}
U7 = {'type': 'ineq',
             'fun' : u7}
# x = minimize(objective, X0, method='trust-constr', options = {'disp':True}).x#, constraints = [ineq_cons], options={'ftol': 1e-9, 'disp': True})

In [140]:
import warnings
warnings.filterwarnings("ignore")

samples = filtered_df.iloc[np.random.choice(len(filtered_df), 197), :8]
results = []
for i in range(len(samples)):
    X0 = np.array(samples.iloc[i])
    res = minimize(objective, X0, method='COBYLA', 
               constraints = [ineq_cons, L0, L1, L2, L3, L4, L5, L6, L7, U0, U1, U2, U3, U4, U5, U6, U7], 
               options = {'verbose':1, 'display':1, 'maxiter':10}) # constraints = [constraint], options = {'verbose':1})#, bounds = bounds)
    results.append((X0, res.x, -res.fun))

In [141]:
sorted(results, key = lambda x: x[2], reverse = True)[:10]

[(array([8.76254164, 2.40324077, 8.77518014, 3.82322874, 4.85104019,
         4.60577037, 8.11653639, 8.55373399]),
  array([ 8.37781964,  2.        , 10.        ,  3.78840952,  4.72647194,
          3.82683827,  9.23497365,  8.51699631]),
  99990.03446809443),
 (array([8.76254164, 2.40324077, 8.77518014, 3.82322874, 4.85104019,
         4.60577037, 8.11653639, 8.55373399]),
  array([ 8.37781964,  2.        , 10.        ,  3.78840952,  4.72647194,
          3.82683827,  9.23497365,  8.51699631]),
  99990.03446809443),
 (array([3.02924659, 2.74524202, 6.29978985, 7.17708261, 5.44099747,
         3.88833632, 4.58883198, 2.70679481]),
  array([2.87590522, 2.        , 7.79436646, 7.1287923 , 5.34349866,
         3.48989421, 5.66564727, 3.70702818]),
  99108.85424540826),
 (array([3.02924659, 2.74524202, 6.29978985, 7.17708261, 5.44099747,
         3.88833632, 4.58883198, 2.70679481]),
  array([2.87590522, 2.        , 7.79436646, 7.1287923 , 5.34349866,
         3.48989421, 5.66564727, 3.70

# Alternative Search

In [127]:
def coordinateAscentQuad(stab_classifier, alt_predictor, initial = [0,0,0,0,0,0,0,0], inc = 0.0000001, max_iter = 10000, confidence = 0.95):
    '''
    Keep increasing in direction until unstable. 
    '''
    M = {}
    c = 9
    for i in range(8):
        for j in range(i, 8):
            M[(i,j)] = c
            c += 1
    def partials(x):
        grad = []
        coef = alt_predictor.coef_
        for i in range(8):
            g = coef[1 + i]
            for j in range(8):
                if i == j:
                    g += 2*x[i]*coef[M[i,i]]
                else:
                    g += coef[M[min([i,j]), max([i,j])]]*x[j]
            grad.append(g)
        return np.array(grad)
    
    x = initial
    old = qr.predict(poly.fit_transform([x]))
    for _ in range(max_iter):
        #for i in range(8):
        y = np.array(x)
        # y[i] += np.sign(alt_predictor.coef_[i])*inc
        # print(y, partials(x)*inc)
        y += partials(x) * inc
        if np.all(y > 2.5) and qr.predict(poly.fit_transform([y])) > old:
            nxt = stab_classifier(torch.from_numpy(poly.fit_transform([y])))[0]
            if nxt > confidence or np.random.rand() > 0.5:
                if nxt <= confidence:
                    print('Non greedy step')
                x = y[:]
                #print(qr.predict(poly.fit_transform([y])) - old)
                old = qr.predict(poly.fit_transform([y]))
                continue
            else:
                break
        
        print('No more feasible directions at iteration', _, '.', qr.predict(poly.fit_transform([x])) - qr.predict(poly.fit_transform([initial])))
        break
    return x

In [147]:
results = []
samples = filtered_df.iloc[np.random.choice(len(filtered_df), 197), :8]
for i in range(len(samples)):
    X0 = np.array(samples.iloc[i])
    result = coordinateAscentQuad(classif, ridge, X0, confidence = .6)
    print(X0, ridge.predict(poly.fit_transform([X0])), classif(torch.from_numpy(poly.fit_transform([X0]))))
    print(ridge.predict(poly.fit_transform([result])), result, classif(torch.from_numpy(poly.fit_transform([result]))))
    results.append((X0, result, ridge.predict(poly.fit_transform([result])), ridge.predict(poly.fit_transform([X0]))))

No more feasible directions at iteration 5845 . [39176.75]
[4.82380255 6.74118993 8.26776052 5.40919866 6.95306475 9.69790705
 2.83948251 5.78381106] [42007.3568367] tensor([[1.0000]], dtype=torch.float64, grad_fn=<SigmoidBackward0>)
[78925.36037793] [4.4090305  2.50080525 9.10555224 5.29160709 6.82691421 9.02145405
 3.21750298 5.75721313] tensor([[1.0000]], dtype=torch.float64, grad_fn=<SigmoidBackward0>)
No more feasible directions at iteration 0 . [0.]
[8.93118239 8.0127891  9.97709212 8.00819807 2.91740485 9.66763027
 4.27111457 6.41234245] [38083.98579968] tensor([[1.0000]], dtype=torch.float64, grad_fn=<SigmoidBackward0>)
[38083.98579968] [8.93118239 8.0127891  9.97709212 8.00819807 2.91740485 9.66763027
 4.27111457 6.41234245] tensor([[1.0000]], dtype=torch.float64, grad_fn=<SigmoidBackward0>)
No more feasible directions at iteration 0 . [0.]
[2.08143832 5.49587179 6.29274552 9.61414548 9.6211931  5.716947
 7.03471605 4.12567358] [49443.8653848] tensor([[1.0000]], dtype=torch.fl

No more feasible directions at iteration 4197 . [40588.5]
[4.20404871 6.23903807 9.53722911 4.87370122 5.62376644 8.01605349
 3.50606934 3.87734483] [48474.36513678] tensor([[1.0000]], dtype=torch.float64, grad_fn=<SigmoidBackward0>)
[86517.2008934] [ 3.85391216  2.50117071 10.18919504  4.7952754   5.47010527  7.30372936
  3.7533133   3.86307765] tensor([[1.0000]], dtype=torch.float64, grad_fn=<SigmoidBackward0>)
No more feasible directions at iteration 0 . [0.]
[5.52565651 7.8627154  7.70078007 3.22432224 9.65843035 8.85661078
 3.97036025 9.9543257 ] [38241.89522611] tensor([[1.0000]], dtype=torch.float64, grad_fn=<SigmoidBackward0>)
[38241.89522611] [5.52565651 7.8627154  7.70078007 3.22432224 9.65843035 8.85661078
 3.97036025 9.9543257 ] tensor([[1.0000]], dtype=torch.float64, grad_fn=<SigmoidBackward0>)
No more feasible directions at iteration 1793 . [25092.75]
[7.55770164 4.50361243 8.21981413 9.34081355 7.3479328  6.0781645
 8.63311071 5.71729505] [59185.27211982] tensor([[1.0000

No more feasible directions at iteration 3825 . [35830.5]
[4.14827727 5.9201598  7.34836891 9.24578232 4.57740199 9.50791698
 7.98383891 4.3721734 ] [46498.18705846] tensor([[1.0000]], dtype=torch.float64, grad_fn=<SigmoidBackward0>)
[80603.6443331] [3.88774277 2.50114229 8.05284827 9.10398149 4.46322015 9.11396327
 8.13186791 4.32635862] tensor([[1.0000]], dtype=torch.float64, grad_fn=<SigmoidBackward0>)
No more feasible directions at iteration 0 . [0.]
[4.63909766 7.2714982  6.86832483 2.26021714 3.85604358 7.69309272
 2.95277771 3.86600877] [40924.22529014] tensor([[1.0000]], dtype=torch.float64, grad_fn=<SigmoidBackward0>)
[40924.22529014] [4.63909766 7.2714982  6.86832483 2.26021714 3.85604358 7.69309272
 2.95277771 3.86600877] tensor([[1.0000]], dtype=torch.float64, grad_fn=<SigmoidBackward0>)
No more feasible directions at iteration 5958 . [35062.25]
[6.18697757 6.56882086 5.81054738 4.11732198 8.37180208 9.82121527
 4.39598059 8.38180369] [39240.30250222] tensor([[1.0000]], dty

[4.20966833 4.82755058 8.33112372 6.05386772 6.45425962 7.42645223
 9.66165679 9.32013025] [58138.74385741] tensor([[1.0000]], dtype=torch.float64, grad_fn=<SigmoidBackward0>)
[70422.04950154] [4.12457293 3.70307509 8.55875774 6.03085477 6.43021108 7.28248928
 9.67803401 9.30537712] tensor([[0.6002]], dtype=torch.float64, grad_fn=<SigmoidBackward0>)
No more feasible directions at iteration 0 . [0.]
[4.85018295 8.39532251 4.51192896 6.54405967 9.2382256  8.89249209
 8.39590231 9.20570753] [32974.22691955] tensor([[1.0000]], dtype=torch.float64, grad_fn=<SigmoidBackward0>)
[32974.22691955] [4.85018295 8.39532251 4.51192896 6.54405967 9.2382256  8.89249209
 8.39590231 9.20570753] tensor([[1.0000]], dtype=torch.float64, grad_fn=<SigmoidBackward0>)
No more feasible directions at iteration 0 . [0.]
[6.25437912 4.24696162 3.97600304 7.53631715 2.00278013 7.8963462
 2.3164338  8.83421963] [48670.73496768] tensor([[1.0000]], dtype=torch.float64, grad_fn=<SigmoidBackward0>)
[48670.73496768] [6.2

No more feasible directions at iteration 2050 . [24182.5]
[4.73444547 4.55713543 4.76768044 3.43126218 6.18119505 7.97143628
 6.97046524 6.40215195] [51791.4423437] tensor([[1.0000]], dtype=torch.float64, grad_fn=<SigmoidBackward0>)
[74743.13982339] [4.58418915 2.50048681 5.33481887 3.43348264 6.13000496 7.72655738
 7.04947791 6.38425255] tensor([[0.6574]], dtype=torch.float64, grad_fn=<SigmoidBackward0>)
No more feasible directions at iteration 0 . [0.]
[4.34722906 7.37824988 2.26491685 9.36257865 8.89383302 6.9966811
 6.60915278 8.03253647] [29782.34066008] tensor([[0.9977]], dtype=torch.float64, grad_fn=<SigmoidBackward0>)
[29782.34066008] [4.34722906 7.37824988 2.26491685 9.36257865 8.89383302 6.9966811
 6.60915278 8.03253647] tensor([[0.9977]], dtype=torch.float64, grad_fn=<SigmoidBackward0>)
No more feasible directions at iteration 2940 . [24419.]
[7.59534288 4.98718601 2.62978363 5.69085127 8.31056625 9.62661298
 9.06865306 9.8931685 ] [40713.13116014] tensor([[1.0000]], dtype=t

No more feasible directions at iteration 1873 . [20208.5]
[5.85864825 4.2843192  6.14918803 6.69545209 5.19060212 9.4347719
 3.21813603 3.40114842] [53314.72121675] tensor([[1.0000]], dtype=torch.float64, grad_fn=<SigmoidBackward0>)
[72033.53855044] [5.7025641  2.50099281 6.55748854 6.65586859 5.13266552 9.17524245
 3.37063917 3.39342084] tensor([[1.0000]], dtype=torch.float64, grad_fn=<SigmoidBackward0>)
No more feasible directions at iteration 2 . [1.25]
[6.39951632 9.79564212 7.35208721 9.51407639 6.46764197 8.22852305
 8.03919005 9.79319301] [35139.58532668] tensor([[0.9987]], dtype=torch.float64, grad_fn=<SigmoidBackward0>)
[35140.0357461] [6.39953456 9.79587122 7.3522088  9.5139997  6.46765067 8.22859748
 8.0391128  9.79312224] tensor([[0.9987]], dtype=torch.float64, grad_fn=<SigmoidBackward0>)
No more feasible directions at iteration 59 . [58.5]
[4.03335766 7.63764846 6.75125065 4.09463841 9.23440151 6.63295844
 4.6558113  5.67161037] [39213.01698667] tensor([[1.0000]], dtype=to

No more feasible directions at iteration 1131 . [16295.5]
[3.05074101 3.76269683 3.68850081 7.14244229 9.71011984 7.10126391
 9.5367193  3.80073645] [56722.98247999] tensor([[1.0000]], dtype=torch.float64, grad_fn=<SigmoidBackward0>)
[72202.79328168] [2.97428316 2.50103601 4.04534168 7.12155259 9.6553422  6.98016349
 9.5564438  3.81266731] tensor([[0.8226]], dtype=torch.float64, grad_fn=<SigmoidBackward0>)
No more feasible directions at iteration 5861 . [43859.]
[5.71548708 7.10260945 8.71101024 8.82660091 8.47732381 7.75343561
 6.88407201 6.98627806] [41374.41854346] tensor([[1.0000]], dtype=torch.float64, grad_fn=<SigmoidBackward0>)
[84217.32274476] [5.31282262 2.50041561 9.56665646 8.59938069 8.29548261 7.05866631
 7.02934684 6.96925644] tensor([[0.9988]], dtype=torch.float64, grad_fn=<SigmoidBackward0>)
No more feasible directions at iteration 1436 . [21933.5]
[3.25988679 4.16643163 6.11047137 9.05088628 3.3246417  7.48755927
 9.4141829  8.73158176] [59887.00392935] tensor([[1.0000

Non greedy step
Non greedy step
[5.0058681  6.10893665 4.29003155 4.94664391 8.75459312 8.26696262
 8.48254088 5.12644021] [39828.71859378] tensor([[1.0000]], dtype=torch.float64, grad_fn=<SigmoidBackward0>)
[69389.8767355] [4.78552155 2.87435239 5.29866846 4.91813498 8.62412995 8.00229716
 8.53974163 5.12534245] tensor([[0.5984]], dtype=torch.float64, grad_fn=<SigmoidBackward0>)
No more feasible directions at iteration 450 . [6165.5]
[8.73286359 2.99412554 8.71650172 5.39791854 9.41658602 9.58566976
 4.35316024 4.88773315] [66893.50596097] tensor([[1.0000]], dtype=torch.float64, grad_fn=<SigmoidBackward0>)
[72643.81225492] [8.6828641  2.50094283 8.79370624 5.38962954 9.39910275 9.51523977
 4.39186969 4.89456629] tensor([[1.0000]], dtype=torch.float64, grad_fn=<SigmoidBackward0>)
No more feasible directions at iteration 5387 . [37682.25]
[8.620305   6.52387304 3.88667836 4.19059542 8.64203901 6.0080067
 6.58945111 9.07213468] [36424.78214145] tensor([[0.9992]], dtype=torch.float64, gra

No more feasible directions at iteration 1719 . [25592.75]
[6.36561609 4.46092386 7.76467428 5.86205955 9.90076986 4.4588183
 6.92587464 5.14131829] [61658.99897107] tensor([[1.0000]], dtype=torch.float64, grad_fn=<SigmoidBackward0>)
[86153.80835399] [6.20428381 2.50037154 8.18003764 5.82915762 9.80571404 4.11784066
 6.96757846 5.16175023] tensor([[1.0000]], dtype=torch.float64, grad_fn=<SigmoidBackward0>)
No more feasible directions at iteration 0 . [0.]
[7.63168609 5.96373284 2.15863472 3.58381163 6.85386144 7.45234988
 8.66903784 3.46879002] [35474.39655556] tensor([[1.0000]], dtype=torch.float64, grad_fn=<SigmoidBackward0>)
[35474.39655556] [7.63168609 5.96373284 2.15863472 3.58381163 6.85386144 7.45234988
 8.66903784 3.46879002] tensor([[1.0000]], dtype=torch.float64, grad_fn=<SigmoidBackward0>)
No more feasible directions at iteration 59 . [58.5]
[4.03335766 7.63764846 6.75125065 4.09463841 9.23440151 6.63295844
 4.6558113  5.67161037] [39213.01698667] tensor([[1.0000]], dtype=to

In [148]:
sorted(results, key = lambda x: x[2], reverse = True)

[(array([3.84330328, 3.11903088, 8.63367165, 8.88974916, 8.81972552,
         3.69938965, 8.75334237, 8.74292895]),
  array([3.79774661, 2.50143278, 8.75329661, 8.87230654, 8.79869793,
         3.59618521, 8.76162157, 8.74398984]),
  array([92073.68628317]),
  array([82869.63799197])),
 (array([8.76254164, 2.40324077, 8.77518014, 3.82322874, 4.85104019,
         4.60577037, 8.11653639, 8.55373399]),
  array([8.76254164, 2.40324077, 8.77518014, 3.82322874, 4.85104019,
         4.60577037, 8.11653639, 8.55373399]),
  array([88072.41631271]),
  array([88072.41631271])),
 (array([4.20404871, 6.23903807, 9.53722911, 4.87370122, 5.62376644,
         8.01605349, 3.50606934, 3.87734483]),
  array([ 3.85391216,  2.50117071, 10.18919504,  4.7952754 ,  5.47010527,
          7.30372936,  3.7533133 ,  3.86307765]),
  array([86517.2008934]),
  array([48474.36513678])),
 (array([6.36561609, 4.46092386, 7.76467428, 5.86205955, 9.90076986,
         4.4588183 , 6.92587464, 5.14131829]),
  array([6.20428

In [145]:
sorted(df['Altitude'], reverse = True)

[87504.62,
 86722.18,
 85846.05,
 85234.52,
 83528.12,
 83442.09,
 83051.34,
 82927.4,
 82061.53,
 81642.69,
 81555.23,
 81538.36,
 80338.0,
 79837.12,
 79274.52,
 78784.51,
 78464.02,
 78285.78,
 78227.84,
 77779.3,
 77117.16,
 76251.66,
 75869.35,
 74482.89,
 74264.98,
 74121.92,
 74057.05,
 73939.99,
 73631.16,
 73446.17,
 73263.34,
 72485.31,
 71832.66,
 71412.88,
 70886.62,
 70830.75,
 68788.69,
 68580.09,
 68412.84,
 67842.77,
 67582.01,
 67541.22,
 65967.9,
 65755.58,
 65196.87,
 64191.2,
 63832.05,
 63545.63,
 63428.37,
 62995.52,
 62259.07,
 61916.03,
 61777.55,
 60676.16,
 60241.04,
 59576.45,
 59528.58,
 59376.38,
 59242.72,
 59079.92,
 58941.78,
 58933.1,
 58280.82,
 58009.89,
 57964.61,
 57829.19,
 57613.95,
 57492.59,
 57476.48,
 57130.39,
 56982.9,
 56945.66,
 56933.59,
 56623.36,
 56583.86,
 56483.61,
 56466.14,
 55940.92,
 55933.92,
 55512.09,
 55372.77,
 55176.56,
 55084.65,
 54923.63,
 54917.07,
 54591.75,
 54481.37,
 54169.0,
 54082.5,
 53435.33,
 53361.03,
 53131.5

In [149]:
best = [3.79774661, 2.50143278, 8.75329661, 8.87230654, 8.79869793,
         3.59618521, 8.76162157, 8.74398984]

In [152]:
classif(torch.from_numpy(poly.fit_transform([best])))

tensor([[0.9999]], dtype=torch.float64, grad_fn=<SigmoidBackward0>)

In [192]:
# Interaction terms help marginally. 
#X_train, X_test = df.iloc[100:300, :-15], df.iloc[300:, :-15]
#X_train, X_test = df.iloc[100:, :-15], df.iloc[:100, :-15]
idxs = np.array(list(range(200, 400)) + list(range(100)))
X_train, X_test = poly.fit_transform(df.iloc[idxs, :-15]), poly.fit_transform(df.iloc[100:200, :-15])
#X_train, X_test = df.iloc[100:300, :-15], df.iloc[300:, :-15]

# Stability level 1: cluster into flight time of 20 and 90.
#Y_train, Y_test = df.iloc[:300, -1] > 90, df.iloc[300:, -1] > 90
#Y_train, Y_test = df.iloc[100:, -1] > 90, df.iloc[:100, -1] > 90
Y_train, Y_test = df.iloc[idxs, -1] > 90, df.iloc[100:200, -1] > 90
#Y_train, Y_test = df.iloc[:300, -1] > 90, df.iloc[300:, -1] > 90

reg = LR(penalty = 'l2')
reg.fit(X_train, Y_train)
sum(reg.predict(X_test) == Y_test)

93

In [162]:
poly

PolynomialFeatures()